# Probabilistic Model Final Project
## Berkan Ottlik and Quentin Chu

In [1]:
import numpy as np
import torch
import plotly.graph_objects as go

from sklearn.linear_model import ARDRegression
from sklearn.metrics import mean_squared_error

from models.reduced_rank_model import ReducedRankModel
from models.neural_network import NeuralNetworkModel
from models.mean_predictor import MeanPredictor
from models.yizi_decoders import continuous_decoder

from utils.get_data import get_data, get_test_train_data, smooth_x, smooth_y
from utils.evaluation import evaluate_recording
from utils.training import train_one_epoch


DATA_PATH = 'raw_data/full_data/'

RECORDING_IDS = np.arange(10)
TOTAL_RECORDINGS = len(RECORDING_IDS) # In total there are 112 recordings
PLOT_RECORDINGS = 3
TIME_BINS = 40
TRAIN_SPLIT = 0.8
TRAINING_ITERATIONS = 20
RANK = 3
MEAN_SUBTRACTED = False
SMOOTHED = True
SMOOTHED_X = True
MODEL = "neural_network"
RANDOM_SEED = 41

In [2]:
X_list, Y_list = get_data(DATA_PATH, RECORDING_IDS)

if SMOOTHED_X:
    X_list = smooth_x(X_list)

if SMOOTHED:
    Y_list = smooth_y(Y_list)

X_train, X_test, Y_train_regular, Y_test_regular, Y_train_mean_subtracted, Y_test_mean_subtracted = get_test_train_data(X_list, Y_list, TRAIN_SPLIT, random_seed=RANDOM_SEED)

Y_train = Y_train_regular
Y_test = Y_test_regular

In [3]:
for t in range(3):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=Y_train_regular[0][t], name='Original'))
    fig.add_trace(go.Scatter(y=Y_train_mean_subtracted[0][t], name='Mean Subtracted'))
    fig.update_layout(title=f'Original, Smoothed, and Mean Subtracted Wheel Speeds, Recording 0, Trial {t}', xaxis_title='Time', yaxis_title='Value')
    fig.show()

# Training

In [4]:
loss_fn = torch.nn.MSELoss()

## Reduced Rank

In [38]:
reduced_rank_model = ReducedRankModel(
    n_recordings=TOTAL_RECORDINGS,
    n_neurons_per_recording=[X_train[i].shape[1] for i in range(TOTAL_RECORDINGS)], 
    n_time_bins=TIME_BINS, 
    rank=RANK
)

if RANDOM_SEED != -1:
    torch.manual_seed(RANDOM_SEED)

optimizer = torch.optim.Adam(reduced_rank_model.parameters(), lr=0.001, weight_decay=0.01)
loss_fn = torch.nn.MSELoss()

train_loss = []
test_loss = []

for training_iteration in range(2001):
    train_loss_iter, test_loss_iter = train_one_epoch(X_train, Y_train, X_test, Y_test, reduced_rank_model, optimizer, loss_fn)

    if training_iteration % 100 == 0:
        print(f'Iteration {training_iteration}, Train Loss {train_loss_iter}, Test Loss {test_loss_iter}')

    train_loss.append(train_loss_iter)
    test_loss.append(test_loss_iter)

Iteration 0, Train Loss [0.8635337352752686, 0.8856847882270813, 0.8476439118385315, 0.8674066662788391, 0.8559505939483643, 0.8637953400611877, 0.847272515296936, 0.8615921139717102, 0.8643249869346619, 0.8323013186454773], Test Loss [0.8607616424560547, 0.8870499730110168, 0.8451713919639587, 0.8613987565040588, 0.853600263595581, 0.8578510284423828, 0.8460714221000671, 0.8614840507507324, 0.8631396293640137, 0.830883264541626]
Iteration 100, Train Loss [0.16941814124584198, 0.1753043532371521, 0.16222088038921356, 0.16273070871829987, 0.16464154422283173, 0.165037602186203, 0.16233989596366882, 0.171702578663826, 0.16543123126029968, 0.16266243159770966], Test Loss [0.16885100305080414, 0.17651231586933136, 0.1634940356016159, 0.16025002300739288, 0.16214804351329803, 0.16396698355674744, 0.16218730807304382, 0.16905157268047333, 0.16769902408123016, 0.1616782397031784]
Iteration 200, Train Loss [0.03308320790529251, 0.037375353276729584, 0.0337732769548893, 0.036787498742341995, 0.

In [27]:
final_loss = []
for rank in range(10):
    reduced_rank_model = ReducedRankModel(
        n_recordings=TOTAL_RECORDINGS,
        n_neurons_per_recording=[X_train[i].shape[1] for i in range(TOTAL_RECORDINGS)], 
        n_time_bins=TIME_BINS, 
        rank=rank
    )

    if RANDOM_SEED != -1:
        torch.manual_seed(RANDOM_SEED)

    optimizer = torch.optim.Adam(reduced_rank_model.parameters(), lr=0.001, weight_decay=0.01)
    loss_fn = torch.nn.MSELoss()

    for training_iteration in range(2001):
        train_loss_iter, test_loss_iter = train_one_epoch(X_train, Y_train, X_test, Y_test, reduced_rank_model, optimizer, loss_fn)

        if training_iteration % 100 == 0:
            print(f'Iteration {training_iteration}, Train Loss {train_loss_iter}, Test Loss {test_loss_iter}')

    final_loss.append(test_loss_iter)

Iteration 0, Train Loss [0.891232430934906, 0.9479613900184631, 0.9473305344581604, 0.9402413964271545, 0.9586836695671082, 0.9234483242034912, 0.8931715488433838, 0.9748676419258118, 0.9525145292282104, 0.9899808764457703], Test Loss [0.8873071074485779, 0.9395264983177185, 0.9537732005119324, 0.9425259828567505, 0.9505452513694763, 0.9263108968734741, 0.8950291275978088, 0.953113853931427, 0.9681713581085205, 0.9927659630775452]
Iteration 100, Train Loss [0.18571674823760986, 0.20226389169692993, 0.21324074268341064, 0.21512675285339355, 0.2272181361913681, 0.20033901929855347, 0.18976804614067078, 0.22622226178646088, 0.2157743275165558, 0.24901354312896729], Test Loss [0.18414267897605896, 0.19813434779644012, 0.22010517120361328, 0.21679696440696716, 0.22418953478336334, 0.20092667639255524, 0.19189372658729553, 0.21434347331523895, 0.22765716910362244, 0.24833610653877258]
Iteration 200, Train Loss [0.04179965704679489, 0.04698282852768898, 0.05416901409626007, 0.0572106093168258

In [28]:
final_loss = np.array(final_loss)
loss_per_rank_0 = final_loss[:, 0]
loss_per_rank_1 = final_loss[:, 1]
loss_per_rank_2 = final_loss[:, 2]
loss_per_rank_3 = final_loss[:, 3]
loss_per_rank_4 = final_loss[:, 4]
loss_per_rank_5 = final_loss[:, 5]
loss_per_rank_6 = final_loss[:, 6]
loss_per_rank_7 = final_loss[:, 7]
loss_per_rank_8 = final_loss[:, 8]
loss_per_rank_9 = final_loss[:, 9]

In [29]:
# plot loss per rank
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(10), y=loss_per_rank_0, name='Session 0'))
# fig.add_trace(go.Scatter(x=np.arange(10), y=loss_per_rank_1))
# fig.add_trace(go.Scatter(x=np.arange(10), y=loss_per_rank_2))
# fig.add_trace(go.Scatter(x=np.arange(10), y=loss_per_rank_3))
# fig.add_trace(go.Scatter(x=np.arange(10), y=loss_per_rank_4))
fig.add_trace(go.Scatter(x=np.arange(10), y=loss_per_rank_5, name='Session 5'))
fig.add_trace(go.Scatter(x=np.arange(10), y=loss_per_rank_6, name='Session 6'))
# fig.add_trace(go.Scatter(x=np.arange(10), y=loss_per_rank_7))
# fig.add_trace(go.Scatter(x=np.arange(10), y=loss_per_rank_8))
# fig.add_trace(go.Scatter(x=np.arange(10), y=loss_per_rank_9))
fig.update_layout(title='Loss per Rank', xaxis_title='Rank', yaxis_title='Test MSE Loss')
fig.show()

In [39]:
train_loss = np.array(train_loss)
test_loss = np.array(test_loss)

fig = go.Figure()
for recording in range(TOTAL_RECORDINGS):
    fig.add_trace(go.Scatter(y=test_loss[:, recording], name=f'{recording} - Test Recording'))
    fig.add_trace(go.Scatter(y=train_loss[:, recording], name=f'{recording} - Train Recording'))
fig.update_layout(title=f'Loss Curve Rank {RANK}', xaxis_title='Iteration', yaxis_title='Loss')
fig.show()

test_loss_reduced_rank = test_loss[-1]

## Neural Network

In [10]:
neural_network_model = NeuralNetworkModel(
    n_recordings=TOTAL_RECORDINGS,
    n_neurons_per_recording=[X_train[i].shape[1] for i in range(TOTAL_RECORDINGS)], 
    n_time_bins=TIME_BINS, 
    width=1000,
    hidden_layers=3,
    rank=RANK
)

if RANDOM_SEED != -1:
    torch.manual_seed(RANDOM_SEED)

optimizer = torch.optim.Adam(neural_network_model.parameters(), lr=0.0001, weight_decay=0.001)
train_loss = []
test_loss = []

for training_iteration in range(51):
    train_loss_iter, test_loss_iter = train_one_epoch(X_train, Y_train, X_test, Y_test, neural_network_model, optimizer, loss_fn)

    if training_iteration % 10 == 0:
        print(f'Iteration {training_iteration}, Train Loss {train_loss_iter}, Test Loss {test_loss_iter}')

    train_loss.append(train_loss_iter)
    test_loss.append(test_loss_iter)

Iteration 0, Train Loss [0.012915035709738731, 0.029783500358462334, 0.030698927119374275, 0.029180098325014114, 0.0350312776863575, 0.016129499301314354, 0.005953098181635141, 0.025206631049513817, 0.016451096162199974, 0.030680108815431595], Test Loss [0.007736602332442999, 0.020557334646582603, 0.031204519793391228, 0.024886751547455788, 0.026911165565252304, 0.013829834759235382, 0.006681143771857023, 0.015417583286762238, 0.020026223734021187, 0.024179590865969658]
Iteration 10, Train Loss [0.004932695999741554, 0.006893796846270561, 0.00822917278856039, 0.010940104722976685, 0.01470231730490923, 0.007645949721336365, 0.004308858420699835, 0.0132354237139225, 0.008648707531392574, 0.013020652346313], Test Loss [0.004216399043798447, 0.00642250943928957, 0.011927109211683273, 0.010702244006097317, 0.01344063226133585, 0.007304628379642963, 0.0054025608114898205, 0.010532788001000881, 0.013084745034575462, 0.011153623461723328]
Iteration 20, Train Loss [0.004915507044643164, 0.00688

In [11]:
train_loss = np.array(train_loss)
test_loss = np.array(test_loss)

fig = go.Figure()
for recording in range(TOTAL_RECORDINGS):
    fig.add_trace(go.Scatter(y=test_loss[:, recording], name=f'{recording} - Test Recording'))
    fig.add_trace(go.Scatter(y=train_loss[:, recording], name=f'{recording} - Train Recording'))
fig.update_layout(title=f'Loss Curve Rank {RANK}', xaxis_title='Iteration', yaxis_title='Loss')
# Show the y axis from 0 to 0.1
fig.update_yaxes(range=[0, 0.04])
fig.show()

test_loss_neural_network = test_loss[-1]

## Continuous Decoder model

In [12]:
Y_pred_continuous_decoder = []
test_loss_continuous_decoder = []

for recording_session in range(TOTAL_RECORDINGS):
    y_pred, evaluation = continuous_decoder(X_train[recording_session], Y_train[recording_session], X_test[recording_session], Y_test[recording_session], time_independent=False)
    Y_pred_continuous_decoder.append(y_pred)
    test_loss_continuous_decoder.append(evaluation[2])

test_loss_continuous_decoder = np.array(test_loss_continuous_decoder)

time-dependent:
Chosen alpha: 10
r2: 0.4048844407507486 corr: 0.6366075045442827 mse: 0.0037442977499152785
Ridge estimator coeff shape: (40, 1000)
time-dependent:
Chosen alpha: 10
r2: 0.6196385316388416 corr: 0.7931238265003442 mse: 0.00594586160930236
Ridge estimator coeff shape: (40, 1440)
time-dependent:
Chosen alpha: 100
r2: 0.3613423411459421 corr: 0.6037203312785178 mse: 0.011845174910779311
Ridge estimator coeff shape: (40, 80)
time-dependent:
Chosen alpha: 10
r2: 0.5207979109392245 corr: 0.7246483166785802 mse: 0.008834624159839544
Ridge estimator coeff shape: (40, 8440)
time-dependent:
Chosen alpha: 100
r2: 0.34792267332078697 corr: 0.591816162275075 mse: 0.013374353626567782
Ridge estimator coeff shape: (40, 1880)
time-dependent:
Chosen alpha: 10
r2: 0.507113053914759 corr: 0.7154432100515467 mse: 0.006345159231634093
Ridge estimator coeff shape: (40, 6880)
time-dependent:
Chosen alpha: 10
r2: 0.4392623680637321 corr: 0.6690876719679009 mse: 0.004577846257350676
Ridge estima

## ARD Regression Model

In [13]:
Y_pred_ARD = []
test_loss_ARD = []
ARD_model = ARDRegression(alpha_1=1e-4, alpha_2=1e-4, lambda_1=1e-4, lambda_2=1e-4)

for recording in range(TOTAL_RECORDINGS):
    X_train_new = X_train[recording].reshape(-1, X_train[recording].shape[1])
    X_test_new = X_test[recording].reshape(-1, X_test[recording].shape[1])
    Y_train_new = Y_train[recording].reshape(-1)
    Y_test_new = Y_test[recording].reshape(-1)

    ARD_model.fit(X_train_new, Y_train_new)
    Y_pred = ARD_model.predict(X_test_new)
    Y_pred_ARD.append(Y_pred)
    
    test_loss = mean_squared_error(Y_test_new, Y_pred)
    print(f'Performance of ARD model on test data: {recording}, Loss {test_loss}')
    test_loss_ARD.append(test_loss)

test_loss_ARD = np.array(test_loss_ARD)

Performance of ARD model on test data: 0, Loss 0.006154119210013087
Performance of ARD model on test data: 1, Loss 0.015055823009818661
Performance of ARD model on test data: 2, Loss 0.018260877309894796
Performance of ARD model on test data: 3, Loss 0.013179598008661158
Performance of ARD model on test data: 4, Loss 0.019953774790003615
Performance of ARD model on test data: 5, Loss 0.012093721606147476
Performance of ARD model on test data: 6, Loss 0.006976589533092136
Performance of ARD model on test data: 7, Loss 0.01730845703773516
Performance of ARD model on test data: 8, Loss 0.020722080320534974
Performance of ARD model on test data: 9, Loss 0.019378954010450342


## Mean Predictor

In [14]:
mean_predictor = MeanPredictor(Y_train)
test_loss_mean_predictor = []

for recording in range(TOTAL_RECORDINGS):
    loss_item = evaluate_recording(recording, X_test[recording], Y_test[recording], mean_predictor, loss_fn, plot_num=0)
    print(f'Performance of zero predictor on training data: {recording}, Loss {loss_item}')
    test_loss_mean_predictor.append(loss_item)

test_loss_mean_predictor = np.array(test_loss_mean_predictor)

Performance of zero predictor on training data: 0, Loss 0.004259638781367352


Performance of zero predictor on training data: 1, Loss 0.006405530499743071
Performance of zero predictor on training data: 2, Loss 0.011840593034430465
Performance of zero predictor on training data: 3, Loss 0.010786176565134337
Performance of zero predictor on training data: 4, Loss 0.013490291922324248
Performance of zero predictor on training data: 5, Loss 0.007057147173683467
Performance of zero predictor on training data: 6, Loss 0.00530523171799835
Performance of zero predictor on training data: 7, Loss 0.010808111729447486
Performance of zero predictor on training data: 8, Loss 0.013081917097458139
Performance of zero predictor on training data: 9, Loss 0.011311714256843979


In [15]:
fig = go.Figure()
fig.add_trace(go.Bar(x=np.arange(TOTAL_RECORDINGS), y=test_loss_mean_predictor, name='Mean Predictor'))
fig.add_trace(go.Bar(x=np.arange(TOTAL_RECORDINGS), y=test_loss_reduced_rank, name='Reduced Rank'))
fig.add_trace(go.Bar(x=np.arange(TOTAL_RECORDINGS), y=test_loss_ARD, name='ARD Model'))
fig.add_trace(go.Bar(x=np.arange(TOTAL_RECORDINGS), y=test_loss_neural_network, name='Neural Network'))
fig.add_trace(go.Bar(x=np.arange(TOTAL_RECORDINGS), y=test_loss_continuous_decoder, name='Continuous Decoder'))
fig.update_layout(title=f'Loss Curve Rank {RANK}', xaxis_title='Recording Session', yaxis_title='Test MSE')
fig.show()